## Tabular training

In [18]:
# Install libraries on first run
#! pip install -q ipynb fastai pathlib pandas

To illustrate the tabular application, we will use the example of the Adult dataset where we have to predict if a person is earning more or less than $50k per year using some general data.

In [2]:
from fastai.tabular.all import *
from pathlib import Path
import pandas as pd

## Variables

In [3]:
yNames = ['Future Year Change']
catNames = []
contNames = [
    'Open',
    'High', 
    'Low', 
    'Close', 
    'Volume', 
    'Dividends', 
    'Stock Splits', 
    'EV/EBIT', 
    'Market Cap', 
    'ROIC'
]

We can download a sample of this dataset with the usual untar_data command:

In [4]:
dataPath = Path()
dataPath.ls()

(#5) [Path('Tutorial'),Path('.DS_Store'),Path('screener.ipynb'),Path('data.csv'),Path('tabular.ipynb')]

Then we can have a look at how the data is structured:

In [5]:
df = pd.read_csv(dataPath/'data.csv')
df.head()

,Date,Ticker,Open,High,Low,Close,Volume,Dividends,Stock Splits,Future Year Change,EV/EBIT,Market Cap,ROIC
0,2019-01-16 00:00:00-05:00,AAPL,36.552888,37.221481,36.533785,36.997025,122278800,0.0,0.0,1.065163,10.403937,5.563575e+11,0.075933
1,2019-01-17 00:00:00-05:00,AAPL,36.820317,37.646508,36.595861,37.216698,119284800,0.0,0.0,1.075703,10.460257,5.596610e+11,0.075524
2,2019-01-18 00:00:00-05:00,AAPL,37.608305,37.699044,37.245354,37.445934,135004000,0.0,0.0,1.049015,10.519028,5.631082e+11,0.075102
3,2019-01-22 00:00:00-05:00,AAPL,37.348034,37.424443,36.443045,36.605419,121576000,0.0,0.0,1.103545,10.303539,5.504686e+11,0.076673
4,2019-01-23 00:00:00-05:00,AAPL,36.808390,37.044787,36.223373,36.753471,92522400,0.0,0.0,1.105161,10.341496,5.526950e+11,0.076391


Some of the columns are continuous (like age) and we will treat them as float numbers we can feed our model directly. Others are categorical (like workclass or education) and we will convert them to a unique index that we will feed to embedding layers. We can specify our categorical and continuous column names, as well as the name of the dependent variable in TabularDataLoaders factory methods:

In [6]:
dls = TabularDataLoaders.from_csv(dataPath/'data.csv', path=dataPath, 
    y_names=yNames,
    cat_names=catNames,
    cont_names=contNames,
    procs = [Categorify, FillMissing, Normalize])

The last part is the list of pre-processors we apply to our data:

* Categorify is going to take every categorical variable and make a map from integer to unique categories, then replace the values by the corresponding index.
* FillMissing will fill the missing values in the continuous variables by the median of existing values (you can choose a specific value if you prefer)
* Normalize will normalize the continuous variables (subtract the mean and divide by the std)

To further expose what’s going on below the surface, let’s rewrite this utilizing fastai’s TabularPandas class. We will need to make one adjustment, which is defining how we want to split our data. By default the factory method above used a random 80/20 split, so we will do the same:

In [7]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

In [8]:
to = TabularPandas(df, procs=[Categorify, FillMissing,Normalize],
    y_names=yNames,
    cat_names = catNames,
    cont_names = contNames,
    splits=splits)

Once we build our TabularPandas object, our data is completely preprocessed as seen below:

In [9]:
to.xs.iloc[:2]

,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,Market Cap,ROIC
22563,0.140109,0.153317,0.147694,0.158993,-0.291245,-0.055841,-0.009184,0.312728,-0.236017,-0.006319
10164,-0.284408,-0.288052,-0.290407,-0.291777,-0.272607,-0.055841,-0.009184,-0.325732,-0.295921,-0.000461


Now we can build our DataLoaders again:

In [10]:
dls = to.dataloaders(bs=64)

The show_batch method works like for every other application:

In [11]:
dls.show_batch()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,Market Cap,ROIC,Future Year Change
0,10.239978,10.532805,10.171579,10.496750,1.943900e+07,-1.257652e-10,8.453505e-13,5.311662,1.350607e+10,0.148729,0.867289
1,275.497622,282.520812,272.738157,279.761322,3.236810e+07,-1.257652e-10,8.453505e-13,25.092832,6.098797e+11,0.031483,0.152192
2,70.665600,71.640598,70.646167,71.334691,2.434200e+07,-1.257652e-10,8.453505e-13,6.489459,3.947662e+11,0.121736,0.697008
3,551.130007,556.900030,547.570009,554.580018,4.309799e+06,-1.257652e-10,8.453505e-13,43.692863,2.370597e+11,0.018081,-0.335804
4,95.599997,98.070003,87.339998,90.679996,6.315099e+06,-1.257652e-10,8.453505e-13,23.706772,2.376723e+10,0.033324,-0.289259
5,214.419998,216.399994,209.169998,210.179993,2.965801e+06,-1.257652e-10,8.453505e-13,97.157769,4.246120e+10,0.008131,0.384004
6,64.800006,65.003468,64.092283,64.233836,5.736801e+06,-1.257652e-10,8.453505e-13,22.758095,8.005270e+10,0.034713,0.229671
7,295.809999,297.730015,277.999999,284.019987,3.096770e+07,-1.257652e-10,8.453505e-13,136.172673,1.092960e+11,0.005801,-0.442258
8,69.230238,70.268519,69.114162,70.084173,3.310800e+07,-1.257652e-10,8.453505e-13,6.353708,3.878459e+11,0.124337,0.674882
9,126.879997,127.379998,120.410006,124.280000,2.913800e+06,-1.257652e-10,8.453505e-13,27.862205,2.006575e+10,0.028354,0.536249


We can define a model using the tabular_learner method. When we define our model, fastai will try to infer the loss function based on our y_names earlier.

Note: Sometimes with tabular data, your y’s may be encoded (such as 0 and 1). In such a case you should explicitly pass y_block = CategoryBlock in your constructor so fastai won’t presume you are doing regression.

In [12]:
learn = tabular_learner(dls, metrics=[rmse, mae])

And we can train that model with the fit_one_cycle method (the fine_tune method won’t be useful here since we don’t have a pretrained model).

In [13]:
learn.fit_one_cycle(50)

epoch,train_loss,valid_loss,_rmse,mae,time
0,0.428250,41.091446,6.410262,0.484511,00:08
1,0.397296,1.235387,1.111480,0.353022,00:07
2,0.376788,14665.791992,121.102318,1.574907,00:07
3,0.345492,427.084167,20.666012,0.566625,00:07
4,0.373656,1.002853,1.001425,0.388823,00:07
5,0.365369,5.484976,2.342003,0.411603,00:07
6,0.402289,2917.595703,54.014778,1.058135,00:08
7,0.313314,435371.281250,659.826965,7.270317,00:07
8,0.318066,62040.488281,249.079391,3.225542,00:07
9,0.361539,4816.038086,69.397690,1.099457,00:07


We can then have a look at some predictions:

In [14]:
learn.show_results()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,Market Cap,ROIC,Future Year Change,Future Year Change_pred
0,-0.534166,-0.534483,-0.532523,-0.534070,-0.186857,-0.055841,-0.009184,-0.368182,-0.480583,0.000187,0.082953,0.437202
1,0.364124,0.352564,0.366166,0.361503,0.059956,-0.055841,-0.009184,1.398911,5.122336,-0.010135,0.294642,0.169348
2,-0.147592,-0.151677,-0.148167,-0.145583,-0.270271,-0.055841,-0.009184,1.439657,-0.218614,-0.010222,-0.018450,-0.178675
3,-0.139966,-0.144591,-0.158558,-0.156862,-0.224587,-0.055841,-0.009184,1.767997,-0.421784,-0.010836,-0.143488,-0.175443
4,-0.554940,-0.556201,-0.553426,-0.554556,-0.256084,-0.055841,-0.009184,-0.452912,-0.496248,0.001654,-0.263291,0.384135
5,-0.023313,0.013213,-0.015533,0.008721,-0.293270,-0.055841,-0.009184,2.648038,-0.442048,-0.011990,-0.247918,-0.208964
6,-0.514763,-0.514650,-0.513250,-0.512507,-0.277426,-0.055841,-0.009184,-0.735211,-0.516540,0.009256,0.011338,0.484943
7,-0.570714,-0.568446,-0.568825,-0.567747,-0.328908,-0.055841,-0.009184,-1.168346,-0.542111,0.057576,1.072874,0.739778
8,-0.390099,-0.386543,-0.391233,-0.391145,0.547314,-0.055841,-0.009184,0.167504,-0.252580,-0.005407,0.494953,0.397084



To get prediction on a new dataframe, you can use the test_dl method of the DataLoaders. That dataframe does not need to have the dependent variable in its column.

In [29]:
predictionTarget = 'PLTR'

from ipynb.fs.full.screener import getTickerData

test_df = getTickerData(predictionTarget)

# Ensure test_df is a DataFrame
if isinstance(test_df, dict):
	test_df = pd.DataFrame([test_df])

dl = learn.dls.test_dl(test_df)

Static Data for PLTR:
  Total Debt: 254915008
  Cash: 4564659200
  Shares Outstanding: 2180649984
Approximated EBIT for PLTR: 396950707.2


In [30]:
prediction = learn.get_preds(dl=dl)
print(f"Predicted {yNames[0]} for {predictionTarget}:")
print(prediction[0][0][0])

Predicted Future Year Change for PLTR:
tensor(377.1684)


Note:
Since machine learning models can’t magically understand categories it was never trained on, the data should reflect this. If there are different missing values in your test data you should address this before training